In [14]:
from datasets import *

In [15]:
def test(model, enc_input, start_symbol):
    # Starting Reference: http://nlp.seas.harvard.edu/2018/04/03/attention.html#greedy-decoding
    enc_outputs, enc_self_attns = model.Encoder(enc_input)
    dec_input = torch.zeros(1, tgt_len).type_as(enc_input.data)
    next_symbol = start_symbol
    for i in range(0, tgt_len):
        dec_input[0][i] = next_symbol
        dec_outputs, _, _ = model.Decoder(dec_input, enc_input, enc_outputs)
        projected = model.projection(dec_outputs)
        prob = projected.squeeze(0).max(dim=-1, keepdim=False)[1]
        next_word = prob.data[i]
        next_symbol = next_word.item()
    return dec_input


In [24]:
enc_inputs, dec_inputs, dec_outputs = make_data()
loader = Data.DataLoader(MyDataSet(enc_inputs, dec_inputs, dec_outputs), 3, True)
enc_inputs, _, _ = next(iter(loader))
model = torch.load('model.pth')

In [28]:
for k in range(3):
    predict_dec_input = test(model, enc_inputs[k].view(1, -1).cuda(), start_symbol=tgt_vocab["S"])
    predict, _, _, _ = model(enc_inputs[k].view(1, -1).cuda(), predict_dec_input)
    predict = predict.data.max(1, keepdim=True)[1]

    print([src_idx2word[int(i)] for i in enc_inputs[k]], '->',
        [idx2word[n.item()] for n in predict.squeeze()])

['后', '排', '功', '能', '控', '制', 'P', 'P', 'P', 'P'] -> ['后', '排', '功', '能', '控', '制', '空', '调', '调', '调']
['副', '驾', '遮', '阳', '板', '化', '妆', '镜', 'P', 'P'] -> ['化', '妆', '镜', '镜', 'E', 'E', 'E', 'E', 'E', 'E']
['前', '排', '座', '椅', '靠', '背', '文', '件', '袋', 'P'] -> ['物', '品', '收', '纳', '纳', '纳', '纳', '纳', 'E', 'E']
